In [ ]:
!pip install -U git+https://github.com/huggingface/transformers.git 

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
!pip install trl
!pip install -U accelerate
!pip install -U transformers
!pip install peft
!pip install utils
!pip install transformers
!pip install -U bitsandbytes

In [ ]:
import pandas as pd

In [ ]:
train_csv=pd.read_csv('/mnt/DATA/Dhanvin/ML_CHALL/student_resource 3/dataset/train.csv')

In [ ]:
train_csv

In [ ]:
train_csv_modified=train_csv.drop(['group_id'],axis=1)

In [ ]:
train_csv_modified.rename(columns = {'entity_name':'question','entity_value':'answer'}, inplace = True)

In [ ]:
train_csv_modified

In [ ]:
def generate_train_ds():
    ds_arr=[]
    for i in range(2110):
        ds_dict={'image_link':train_csv_modified['image_link'][i],
                  'question':train_csv_modified['question'][i], 
                  'answer':train_csv_modified['answer'][i]}
        ds_arr.append(ds_dict)
    return ds_arr

def generate_test_ds():
    ds_arr_1=[]
    for i in range(528):
        ds_dict={'image_link':train_csv_modified['image_link'][63325+i],
                  'question':train_csv_modified['question'][63325+i], 
                  'answer':train_csv_modified['answer'][63325+i]}
        ds_arr_1.append(ds_dict)
    return ds_arr_1

train_ds=generate_train_ds()
eval_ds=generate_test_ds()


In [ ]:
train_ds[0]

In [ ]:
from transformers import PaliGemmaProcessor, PaliGemmaForConditionalGeneration, BitsAndBytesConfig, Trainer
from PIL import Image
import requests
import torch

In [ ]:
# from PIL import Image
# from io import BytesIO
# import requests
# # import urlopen
# # Image.open(urlopen(train_csv['image_link'][0]))
# for i in range(len(train_ds)):
#     try:

#         response = requests.get(train_ds[i]['image_link'])
#         img = Image.open(BytesIO(response.content))
#     except:
#         train_ds.remove(train_ds[i])




In [ ]:
from huggingface_hub import login

# Read the token from the file
with open("token.txt", "r") as file:
    hf_token = file.read().strip()

# Log in to Hugging Face using the token
login(token=hf_token)


In [ ]:
model_id = "google/paligemma-3b-pt-224"

In [ ]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
device="cuda"
model = PaliGemmaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=bnb_config,
).to(device)

processor = PaliGemmaProcessor.from_pretrained(model_id)

In [ ]:
image_token = processor.tokenizer.convert_tokens_to_ids("<image>")

In [ ]:
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

In [ ]:
# def collate_fn(examples):
#   texts = ['Extract'+example["question"]+' from the image' for example in examples]
#   images = [Image.open(BytesIO(requests.get(example['image_link']).content)).convert("RGB").resize((224,224)) for example in examples]
#   tokens = processor(text=texts, images=images,
#                     return_tensors="pt", padding="longest",
#                     tokenize_newline_separately=False)
#   labels = tokens["input_ids"]
#   labels[labels == image_token] = -100
#   tokens["labels"] = labels
#   return tokens
import numpy as np
def collate_fn(examples):
    texts=[]
    labels=[]
    images=[]
    for example in examples:
        try:

            img = Image.open(BytesIO(requests.get(example['image_link']).content)).convert("RGB").resize((224,224))
            np.array(img)
            texts.append('Extract'+example["question"]+' from the image')
            labels.append(example['answer'])
        
            images.append(Image.open(BytesIO(requests.get(example['image_link']).content)).convert("RGB").resize((224,224)))
            
        except:
            continue

    tokens = processor(text=texts, images=images, suffix=labels,
                            return_tensors="pt", padding="longest",
                            tokenize_newline_separately=False)
    tokens = tokens.to(device)
    return tokens
        

In [ ]:
from peft import get_peft_model, LoraConfig

In [ ]:
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)

In [ ]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


In [ ]:
from transformers import TrainingArguments

In [ ]:
args=TrainingArguments(
            num_train_epochs=2,
            per_device_train_batch_size=2,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            learning_rate=2e-5,
            weight_decay=1e-6,
            adam_beta2=0.999,
            logging_steps=20,
            optim="adamw_hf",
            save_strategy="steps",
            save_steps=50,
            push_to_hub=False,
            save_total_limit=1,
            remove_unused_columns=False,
            output_dir='./model_checkpoints',
            dataloader_pin_memory=False
        )

In [ ]:
from trl import SFTTrainer


In [ ]:
trainer = SFTTrainer(
    model=model,
    args=args,

    dataset_text_field="text",
    peft_config=lora_config,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=collate_fn,
    dataset_kwargs={"skip_prepare_dataset": True}
)

In [ ]:
trainer.train()